# Your Hands-On Guide to Delta Tables (in Python)

In [1]:
from delta import *
from pyspark.sql import SparkSession

builder = SparkSession.builder.appName('delta-tutorial').config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/06/10 12:01:35 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.235 instead (on interface wlp5s0)
23/06/10 12:01:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aadi/.ivy2/cache
The jars for the packages stored in: /home/aadi/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-78234d44-7fcc-43a7-85d1-77c3055c2d82;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 101ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |

23/06/10 12:01:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/10 12:01:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [114]:
# load vanilla spark table
sdf = spark.read.load(
  '/storage/data/airline_2m.csv' ,
  format='com.databricks.spark.csv',
  header='true',
  inferSchema='true'
).select(['FlightDate', 'Reporting_Airline', 'Flight_Number_Reporting_Airline','Origin', 'Dest', 'DepTime', 'DepDelay', 'ArrTime', 'ArrDelay' ])

# save as a delta table
sdf.write.format('delta').mode('overwrite').save('/storage/data/airline_2m.delta')

23/06/10 12:36:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 94.30% for 8 writers
23/06/10 12:36:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 83.83% for 9 writers
23/06/10 12:36:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 75.44% for 10 writers
23/06/10 12:36:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 68.58% for 11 writers
23/06/10 12:36:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 62.87% for 12 writers


23/06/10 12:36:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 68.58% for 11 writers
23/06/10 12:36:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 75.44% for 10 writers
23/06/10 12:36:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 83.83% for 9 writers
23/06/10 12:36:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,012,583,616 bytes) of heap memory
Scaling row group sizes to 94.30% for 8 writers


In [119]:
delta_table = DeltaTable.forPath(spark, '/storage/data/airline_2m.delta/')

In [21]:
# inspecting what we just wrote
import os

os.listdir('/storage/data/airline_2m.delta/')
# bunch of individual files and a folder called "_delta_log"

['part-00008-37835621-905f-481b-bee1-f2fbde02595c-c000.snappy.parquet',
 '.part-00002-f79474ae-503d-4914-8aff-54af17da6de2-c000.snappy.parquet.crc',
 '.part-00006-9cdf358f-5ff7-4c45-9314-09aeeb770ab4-c000.snappy.parquet.crc',
 '.part-00001-da20e657-cbd5-4ca0-b7bf-f862a6083d2b-c000.snappy.parquet.crc',
 'part-00007-3e4232fc-fbd4-4cce-84fb-7a862aff9928-c000.snappy.parquet',
 '.part-00010-bff9f938-783e-490b-b462-636415a9f94d-c000.snappy.parquet.crc',
 '.part-00009-6c30192d-45cc-4819-b3ce-3e6c4de1c218-c000.snappy.parquet.crc',
 'part-00004-016a4d0d-6026-4cff-9c53-8a4af7c32241-c000.snappy.parquet',
 'part-00005-3885ce51-666c-4fb2-a0ad-a12c90ec095a-c000.snappy.parquet',
 '_delta_log',
 'part-00003-9aec4e25-09dd-4f1d-8bff-e6c7c368ae7d-c000.snappy.parquet',
 '.part-00008-37835621-905f-481b-bee1-f2fbde02595c-c000.snappy.parquet.crc',
 'part-00006-9cdf358f-5ff7-4c45-9314-09aeeb770ab4-c000.snappy.parquet',
 'part-00009-6c30192d-45cc-4819-b3ce-3e6c4de1c218-c000.snappy.parquet',
 '.part-00003-9aec4

In [22]:
# _delta_log
os.listdir('/storage/data/airline_2m.delta/_delta_log/')
# a json and a .crc file, crc files are checksums added to prevent corruption if parquet is corrupted in-flight

['00000000000000000000.json', '.00000000000000000000.json.crc']

logs hold:
copy-paste:
Whenever a user performs an operation to modify a table (such as an INSERT, UPDATE or DELETE), Delta Lake breaks that operation down into a series of discrete steps composed of one or more of the actions below.

Add file - adds a data file.
Remove file - removes a data file.
Update metadata - Updates the table’s metadata (e.g., changing the table’s name, schema or partitioning).
Set transaction - Records that a structured streaming job has committed a micro-batch with the given ID.
Change protocol - enables new features by switching the Delta Lake transaction log to the newest software protocol.
Commit info - Contains information around the commit, which operation was made, from where and at what time.


WHen table is created, table's transaciton log automatically created in `_delta_log`. Each change is recorded as an atomic commit in the transaction log

In [23]:
import json

with open('/storage/data/airline_2m.delta/_delta_log/00000000000000000000.json', 'r') as json_file:
    for line in json_file:
      json_object = json.loads(line) 
      print(json.dumps(json_object, indent=2))
  

{
  "commitInfo": {
    "timestamp": 1686413205823,
    "operation": "WRITE",
    "operationParameters": {
      "mode": "Overwrite",
      "partitionBy": "[]"
    },
    "isolationLevel": "Serializable",
    "isBlindAppend": false,
    "operationMetrics": {
      "numFiles": "12",
      "numOutputRows": "2000000",
      "numOutputBytes": "20619620"
    },
    "engineInfo": "Apache-Spark/3.3.2 Delta-Lake/2.3.0",
    "txnId": "4f5656f9-d9d2-404e-bf11-722a1349387b"
  }
}
{
  "protocol": {
    "minReaderVersion": 1,
    "minWriterVersion": 2
  }
}
{
  "metaData": {
    "id": "609a6c9c-5c5d-4844-9276-49201985b8f1",
    "format": {
      "provider": "parquet",
      "options": {}
    },
    "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"FlightDate\",\"type\":\"timestamp\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Reporting_Airline\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Origin\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\"

In [126]:
sdf.filter('Origin="JFK" and FlightDate = "2017-07-26"').show() # filter for flights leaving JFK on the 26th July, 2017

+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+
|         FlightDate|Reporting_Airline|Flight_Number_Reporting_Airline|Origin|Dest|DepTime|DepDelay|ArrTime|ArrDelay|
+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+
|2017-07-26 00:00:00|               AA|                              9|   JFK| SFO|    657|    -3.0|    947|   -33.0|
|2017-07-26 00:00:00|               DL|                           2051|   JFK| SJU|   2036|     7.0|     17|   -26.0|
|2017-07-26 00:00:00|               B6|                           1089|   JFK| MCO|   1340|    21.0|   1638|    27.0|
+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+



In [136]:
# create new table for upsert
sdf.dtypes

updates = [
    (datetime.strptime('2017-07-26', '%Y-%m-%d'), 'AA',9, 'JFK', 'SFO', 756, -4.0, 1117, 0.0), # update existing entry
    (datetime.strptime('2017-07-26', '%Y-%m-%d'), 'DL',1368, 'JFK', 'MIA', 1107, 1.0, 1421, 0.0), # new entry
]
(updates_table := spark.createDataFrame(updates, schema=sdf.schema)).show()

+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+
|         FlightDate|Reporting_Airline|Flight_Number_Reporting_Airline|Origin|Dest|DepTime|DepDelay|ArrTime|ArrDelay|
+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+
|2017-07-26 00:00:00|               AA|                              9|   JFK| SFO|    756|    -4.0|   1117|     0.0|
|2017-07-26 00:00:00|               DL|                           1368|   JFK| MIA|   1107|     1.0|   1421|     0.0|
+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+



In [144]:
# perform upsert
(delta_table.alias('current_data')
  .merge(
      source=updates_table.alias('new_data'), 
      condition=F.expr('current_data.FlightDate = new_data.FlightDate and new_data.Origin = current_data.Origin and current_data.Dest = new_data.Dest and  current_data.Reporting_Airline = new_data.Reporting_Airline and current_data.Flight_Number_Reporting_Airline = new_data.Flight_Number_Reporting_Airline'))
  .whenMatchedUpdate(set = {
    'DepTime': F.col('new_data.DepTime'),
    'DepDelay': F.col('new_data.DepDelay'),
    'ArrTime': F.col('new_data.ArrTime'),
    'ArrDelay': F.col('new_data.ArrDelay'),
  })
  # .whenNotMatchedInsert(values = {
  #   'FlightDate': 'new_data.FlightDate',
  #   'Reporting_Airline': 'new_data.Reporting_Airline',
  #   'Flight_Number_Reporting_Airline': 'new_data.Flight_Number_Reporting_Airline',
  #   'DepTime': 'new_data.DepTime',
  #   'DepDelay': 'new_data.DepDelay',
  #   'ArrTime': 'new_data.ArrTime',
  #   'ArrDelay': 'new_data.ArrDelay',
  # })
  .whenNotMatchedInsertAll()
  .execute()

  # can have any number of whenMatched (at most one update and one delete action)
  # update in merge only updates sepcified columns
  # multiple whenMatched executes in order specified
  # to update all columns of target dleta table use whenMatched(...).updateAll()


  # whenNotMatched when source doesn't match target
  # can have ONLY the insert action, any unspecified columns assume null
  # each whenNotMatched can have optional conditoin, 
  # see https://docs.delta.io/latest/delta-update.html#language-python
)

In [145]:
delta_table.toDF().filter('Origin="JFK" and FlightDate = "2017-07-26"').show()

+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+
|         FlightDate|Reporting_Airline|Flight_Number_Reporting_Airline|Origin|Dest|DepTime|DepDelay|ArrTime|ArrDelay|
+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+
|2017-07-26 00:00:00|               DL|                           2051|   JFK| SJU|   2036|     7.0|     17|   -26.0|
|2017-07-26 00:00:00|               B6|                           1089|   JFK| MCO|   1340|    21.0|   1638|    27.0|
|2017-07-26 00:00:00|               DL|                           1368|   JFK| MIA|   1107|     1.0|   1421|     0.0|
|2017-07-26 00:00:00|               AA|                              9|   JFK| SFO|    756|    -4.0|   1117|     0.0|
+-------------------+-----------------+-------------------------------+------+----+-------+--------+-------+--------+

